In [23]:
import numpy as np
import pandas as pd
import os
import json
from datetime import datetime, timedelta
import time
from typing import Dict, List, Tuple, Optional
import warnings
warnings.filterwarnings('ignore')
import tkinter as tk
from tkinter import filedialog

# For visualization
try:
    from bokeh.plotting import figure, show, output_file, save
    from bokeh.layouts import column, row
    from bokeh.models import ColumnDataSource, HoverTool
    from bokeh.io import curdoc
    BOKEH_AVAILABLE = True
except ImportError:
    BOKEH_AVAILABLE = False
    print("Bokeh not available. Install with: pip install bokeh")

class DataLoader:

    def __init__(self):
        self.data = None
        self.processed_data = None

    def upload_file(self):
        """Upload Excel/CSV file using native file dialog"""
        print("Opening file dialog to select data file...")
        print("Supported formats: Excel (.xlsx, .xls) and CSV (.csv)")
        print("\n" + "="*60)
        
        # Create a root window and hide it
        root = tk.Tk()
        root.withdraw()
        
        # Open file dialog
        filename = filedialog.askopenfilename(
            title="Select parking data file",
            filetypes=[
                ("Excel files", "*.xlsx *.xls"),
                ("CSV files", "*.csv"),
                ("All files", "*.*")
            ]
        )
        
        # Destroy the root window
        root.destroy()
        
        if not filename:
            print("No file selected. Will generate sample data for demonstration.")
            return None
            
        print(f"File selected successfully: {os.path.basename(filename)}")
        return filename

    def load_data(self, file_path: str = None) -> pd.DataFrame:
        # Try to get file from user if not provided
        if not file_path:
            file_path = self.upload_file()
        
        if file_path and os.path.exists(file_path):
            print(f"Loading data from {file_path}")
            try:
                if file_path.endswith('.xlsx') or file_path.endswith('.xls'):
                    self.data = pd.read_excel(file_path)
                    print(f" Successfully loaded Excel file with {len(self.data)} rows")
                else:
                    self.data = pd.read_csv(file_path)
                    print(f" Successfully loaded CSV file with {len(self.data)} rows")

                # Display column information
                print(f" Columns found: {list(self.data.columns)}")

            except Exception as e:
                print(f" Error loading file: {e}")
                print("Falling back to sample data generation...")
                self.data = self.generate_sample_data()
        else:
            print("No data file found. Generating sample data...")
            self.data = self.generate_sample_data()

        return self.preprocess_data()

    def generate_sample_data(self) -> pd.DataFrame:
        """Generate realistic sample data for 14 parking spaces over 73 days"""
        np.random.seed(42)

        # Parameters
        n_spaces = 14
        n_days = 73
        n_timepoints = 18  # 8:00 AM to 4:30 PM (30 min intervals)

        data = []

        # Generate base coordinates for 14 parking spaces in a city grid
        base_coords = [
            (40.7589, -73.9851), (40.7614, -73.9776), (40.7505, -73.9934),
            (40.7580, -73.9855), (40.7590, -73.9845), (40.7600, -73.9835),
            (40.7610, -73.9825), (40.7620, -73.9815), (40.7630, -73.9805),
            (40.7640, -73.9795), (40.7650, -73.9785), (40.7660, -73.9775),
            (40.7670, -73.9765), (40.7680, -73.9755)
        ]

        for day in range(n_days):
            # Special days (weekends, holidays)
            is_special_day = (day % 7 in [5, 6]) or (day in [10, 25, 45, 60])

            for timepoint in range(n_timepoints):
                # Time from 8:00 AM to 4:30 PM
                hour = 8 + (timepoint * 0.5)
                timestamp = pd.Timestamp(f"2024-01-01") + pd.Timedelta(days=day, hours=hour)

                for space_id in range(n_spaces):
                    lat, lon = base_coords[space_id]
                    # Add small random variation
                    lat += np.random.normal(0, 0.001)
                    lon += np.random.normal(0, 0.001)

                    # Capacity varies by space
                    capacity = np.random.randint(20, 100)

                    # Occupancy patterns (higher during business hours)
                    base_occupancy = 0.3 + 0.5 * np.sin((timepoint / n_timepoints) * np.pi)
                    if is_special_day:
                        base_occupancy *= 0.7  # Lower on special days

                    occupancy = int(capacity * base_occupancy * (0.8 + 0.4 * np.random.random()))
                    occupancy = min(occupancy, capacity)

                    # Queue length
                    queue_prob = max(0, (occupancy / capacity - 0.8) * 5)
                    queue_length = np.random.poisson(queue_prob)

                    # Vehicle type
                    vehicle_types = ['car', 'bike', 'truck']
                    vehicle_weights = [0.7, 0.2, 0.1]
                    vehicle_type = np.random.choice(vehicle_types, p=vehicle_weights)

                    # Traffic level
                    traffic_level = np.random.uniform(0.1, 1.0)
                    if 9 <= hour <= 11 or 17 <= hour <= 19:  # Rush hours
                        traffic_level = np.random.uniform(0.7, 1.0)

                    data.append({
                        'timestamp': timestamp,
                        'space_id': space_id,
                        'latitude': lat,
                        'longitude': lon,
                        'capacity': capacity,
                        'occupancy': occupancy,
                        'queue_length': queue_length,
                        'vehicle_type': vehicle_type,
                        'traffic_level': traffic_level,
                        'is_special_day': is_special_day,
                        'day': day,
                        'timepoint': timepoint
                    })

        return pd.DataFrame(data)

    def preprocess_data(self) -> pd.DataFrame:
        """Preprocess the data for modeling"""
        df = self.data.copy()

        print(f" Available columns in your data: {list(df.columns)}")
        print(f" Data shape: {df.shape}")
        print(f" First few rows:")
        print(df.head())

        # Create column mapping for flexibility
        column_mapping = self.create_column_mapping(df.columns)
        print(f" Column mapping: {column_mapping}")

        # Apply column mapping
        df = df.rename(columns=column_mapping)

        # Add missing columns with default values if needed
        required_columns = ['space_id', 'occupancy', 'capacity', 'queue_length', 
                          'vehicle_type', 'traffic_level', 'is_special_day', 'latitude', 'longitude']

        for col in required_columns:
            if col not in df.columns:
                print(f"  Missing column '{col}', adding default values")
                if col == 'space_id':
                    df[col] = range(len(df))
                elif col in ['occupancy', 'capacity']:
                    df[col] = 50  # Default parking capacity/occupancy
                elif col == 'queue_length':
                    df[col] = 0
                elif col == 'vehicle_type':
                    df[col] = 'car'
                elif col == 'traffic_level':
                    df[col] = 0.5
                elif col == 'is_special_day':
                    df[col] = False
                elif col in ['latitude', 'longitude']:
                    df[col] = 40.7589 if col == 'latitude' else -73.9851  # Default NYC coordinates

        # Handle timestamp creation from separate date and time columns
        if 'date' in df.columns and 'time' in df.columns:
            print(" Combining date and time columns into timestamp")
            try:
                # Handle different date/time formats
                df['date'] = pd.to_datetime(df['date'], errors='coerce')
                df['time'] = pd.to_datetime(df['time'], format='%H:%M:%S', errors='coerce').dt.time

                # Combine date and time
                df['timestamp'] = pd.to_datetime(df['date'].dt.date.astype(str) + ' ' + df['time'].astype(str))
                print(f" Successfully created {df['timestamp'].notna().sum()} timestamps")
            except Exception as e:
                print(f"  Error combining date/time: {e}, using date column only")
                df['timestamp'] = pd.to_datetime(df['date'], errors='coerce')
        elif 'timestamp' in df.columns:
            df['timestamp'] = pd.to_datetime(df['timestamp'])
        else:
            # Create default timestamps
            print("  No timestamp column found, creating default timestamps")
            df['timestamp'] = pd.date_range(start='2024-01-01 08:00:00', periods=len(df), freq='30min')

        # Create derived features
        df['occupancy_rate'] = df['occupancy'] / df['capacity']
        df['hour'] = df['timestamp'].dt.hour if 'timestamp' in df.columns else 12
        df['day_of_week'] = df['timestamp'].dt.dayofweek if 'timestamp' in df.columns else 1

        # Handle traffic conditions - convert text to numeric if needed
        if 'traffic_level' in df.columns and df['traffic_level'].dtype == 'object':
            print(" Converting traffic conditions to numeric values")
            traffic_mapping = {
                'low': 0.2, 'light': 0.2, 'minimal': 0.2, 'clear': 0.1,
                'medium': 0.5, 'moderate': 0.5, 'normal': 0.5, 'average': 0.5,
                'high': 0.8, 'heavy': 0.8, 'congested': 0.8, 'busy': 0.7,
                'very high': 1.0, 'severe': 1.0, 'gridlock': 1.0, 'jam': 0.9
            }
            df['traffic_level'] = df['traffic_level'].str.lower().str.strip().map(traffic_mapping).fillna(0.5)
            print(f" Converted traffic conditions: {df['traffic_level'].unique()}")
        elif 'traffic_level' not in df.columns:
            df['traffic_level'] = 0.5

        # Vehicle type encoding - handle various formats
        if 'vehicle_type' in df.columns and df['vehicle_type'].dtype == 'object':
            print(" Encoding vehicle types")
            vehicle_type_weights = {
                'car': 1.0, 'cars': 1.0, 'automobile': 1.0, 'auto': 1.0,
                'bike': 0.5, 'bicycle': 0.5, 'motorcycle': 0.7, 'motorbike': 0.7,
                'truck': 1.5, 'van': 1.3, 'bus': 2.0, 'lorry': 1.5,
                'suv': 1.2, 'sedan': 1.0, 'hatchback': 0.9, 'coupe': 1.0
            }
            df['vehicle_weight'] = df['vehicle_type'].str.lower().str.strip().map(vehicle_type_weights).fillna(1.0)
            print(f" Encoded vehicle types: {df['vehicle_type'].unique()}")
        else:
            df['vehicle_weight'] = 1.0

        # Handle special day column
        if 'is_special_day' in df.columns:
            # Convert various formats to boolean
            if df['is_special_day'].dtype == 'object':
                df['is_special_day'] = df['is_special_day'].str.lower().isin(['true', 'yes', '1', 'y'])
            elif df['is_special_day'].dtype in ['int64', 'float64']:
                df['is_special_day'] = df['is_special_day'] == 1

        # Vehicle type encoding
        vehicle_type_weights = {'car': 1.0, 'bike': 0.5, 'truck': 1.5}
        df['vehicle_weight'] = df['vehicle_type'].map(vehicle_type_weights).fillna(1.0)

        # Normalize features
        df['queue_length_norm'] = df['queue_length'] / (df['queue_length'].max() + 1)
        df['traffic_level_norm'] = df['traffic_level']

        print(f" Data preprocessing completed. Final shape: {df.shape}")
        self.processed_data = df
        return df

    def create_column_mapping(self, columns):
        """Create mapping from user's column names to expected column names"""
        mapping = {}
        columns_lower = [col.lower() for col in columns]

        # Exact mapping for your 12 columns
        exact_mapping = {
            'id': 'space_id',
            'systemcodenumber': 'space_id', 
            'capacity': 'capacity',
            'latitude': 'latitude',
            'longitude': 'longitude', 
            'occupancy': 'occupancy',
            'vehicletype': 'vehicle_type',
            'trafficconditionnearby': 'traffic_level',
            'queuelength': 'queue_length',
            'isspecialday': 'is_special_day',
            'lastupdateddate': 'date',
            'lastupdatedtime': 'time'
        }

        # Apply exact mapping for your columns
        for original_col in columns:
            col_lower = original_col.lower().replace(' ', '').replace('_', '')
            if col_lower in exact_mapping:
                mapping[original_col] = exact_mapping[col_lower]

        return mapping

class PricingModel:
    """Base class for pricing models"""

    def __init__(self, base_price: float = 10.0):
        self.base_price = base_price
        self.price_history = {}

    def calculate_price(self, features: Dict) -> float:
        """Calculate price based on features - to be implemented by subclasses"""
        raise NotImplementedError

    def update_price_history(self, space_id: int, price: float, timestamp: str = None):
        """Update price history for a space"""
        if space_id not in self.price_history:
            self.price_history[space_id] = []

        if timestamp is None:
            timestamp = datetime.now().isoformat()

        self.price_history[space_id].append({
            'timestamp': timestamp,
            'price': price
        })

    def get_current_price(self, space_id: int) -> float:
        """Get current price for a space"""
        if space_id in self.price_history and self.price_history[space_id]:
            return self.price_history[space_id][-1]['price']
        return self.base_price

class LinearPricingModel(PricingModel):
    """Model 1: Baseline Linear Model"""

    def __init__(self, base_price: float = 10.0, alpha: float = 0.5):
        super().__init__(base_price)
        self.alpha = alpha

    def calculate_price(self, features: Dict) -> float:
        """
        Price(t+1) = Price(t) + α * (Occupancy / Capacity)
        """
        space_id = features.get('space_id', 0)
        current_price = self.get_current_price(space_id)
        occupancy_rate = features.get('occupancy_rate', 0)

        price_adjustment = self.alpha * occupancy_rate
        new_price = current_price + price_adjustment

        # Bound the price
        new_price = max(self.base_price * 0.5, min(new_price, self.base_price * 2.0))

        return new_price

class DemandBasedPricingModel(PricingModel):
    """Model 2: Demand-Based Price Function"""

    def __init__(self, base_price: float = 10.0, 
                 alpha: float = 2.0, beta: float = 1.5, gamma: float = 0.5, 
                 delta: float = 0.3, epsilon: float = 0.2, lambda_param: float = 0.8):
        super().__init__(base_price)
        self.alpha = alpha
        self.beta = beta
        self.gamma = gamma
        self.delta = delta
        self.epsilon = epsilon
        self.lambda_param = lambda_param

    def calculate_demand(self, features: Dict) -> float:
        """
        Demand = α*(Occupancy/Capacity) + β*QueueLength - γ*Traffic + δ*IsSpecialDay + ε*VehicleTypeWeight
        """
        occupancy_rate = features.get('occupancy_rate', 0)
        queue_length_norm = features.get('queue_length_norm', 0)
        traffic_level_norm = features.get('traffic_level_norm', 0)
        is_special_day = features.get('is_special_day', False)
        vehicle_weight = features.get('vehicle_weight', 1.0)

        demand = (self.alpha * occupancy_rate + 
                 self.beta * queue_length_norm - 
                 self.gamma * traffic_level_norm + 
                 self.delta * (1 if is_special_day else 0) + 
                 self.epsilon * vehicle_weight)

        return demand

    def calculate_price(self, features: Dict) -> float:
        """
        Price(t) = BasePrice * (1 + λ * NormalizedDemand)
        """
        demand = self.calculate_demand(features)

        # Normalize demand to [-1, 1] range
        normalized_demand = np.tanh(demand / 3.0)

        new_price = self.base_price * (1 + self.lambda_param * normalized_demand)

        # Bound the price
        new_price = max(self.base_price * 0.5, min(new_price, self.base_price * 2.0))

        return new_price

class CompetitivePricingModel(DemandBasedPricingModel):
    """Model 3: Competitive Pricing Model"""

    def __init__(self, base_price: float = 10.0, **kwargs):
        super().__init__(base_price, **kwargs)
        self.competitor_radius = 0.01  # ~1km in lat/lon degrees

    def calculate_distance(self, lat1: float, lon1: float, lat2: float, lon2: float) -> float:
        """Calculate Euclidean distance between two points"""
        return np.sqrt((lat1 - lat2)**2 + (lon1 - lon2)**2)

    def find_competitors(self, features: Dict, all_spaces_data: List[Dict]) -> List[Dict]:
        """Find nearby competitors"""
        current_lat = features.get('latitude', 0)
        current_lon = features.get('longitude', 0)
        current_space_id = features.get('space_id', 0)

        competitors = []
        for space_data in all_spaces_data:
            if space_data.get('space_id') == current_space_id:
                continue

            distance = self.calculate_distance(
                current_lat, current_lon,
                space_data.get('latitude', 0), space_data.get('longitude', 0)
            )

            if distance <= self.competitor_radius:
                competitors.append({
                    'space_id': space_data.get('space_id'),
                    'distance': distance,
                    'occupancy_rate': space_data.get('occupancy_rate', 0),
                    'price': self.get_current_price(space_data.get('space_id', 0))
                })

        return competitors

    def calculate_price(self, features: Dict, all_spaces_data: List[Dict] = None) -> float:
        """Calculate price considering competition"""
        # Start with demand-based price
        base_price = super().calculate_price(features)

        if not all_spaces_data:
            return base_price

        competitors = self.find_competitors(features, all_spaces_data)

        if not competitors:
            return base_price

        # Analyze competition
        avg_competitor_price = np.mean([comp['price'] for comp in competitors])
        avg_competitor_occupancy = np.mean([comp['occupancy_rate'] for comp in competitors])
        current_occupancy_rate = features.get('occupancy_rate', 0)

        # Competitive adjustment
        competitive_factor = 1.0

        # If we're full and competitors are cheaper, reduce price
        if current_occupancy_rate > 0.9 and avg_competitor_price < base_price:
            competitive_factor = 0.9

        # If competitors are expensive and we have capacity, increase price
        elif current_occupancy_rate < 0.7 and avg_competitor_price > base_price:
            competitive_factor = 1.1

        # If competitors are full but we have capacity, increase price more
        elif current_occupancy_rate < 0.5 and avg_competitor_occupancy > 0.8:
            competitive_factor = 1.2

        adjusted_price = base_price * competitive_factor

        # Bound the price
        adjusted_price = max(self.base_price * 0.5, min(adjusted_price, self.base_price * 2.0))

        return adjusted_price

    def suggest_rerouting(self, features: Dict, all_spaces_data: List[Dict] = None) -> Optional[Dict]:
        """Suggest rerouting if current lot is overburdened"""
        if not all_spaces_data:
            return None

        current_occupancy_rate = features.get('occupancy_rate', 0)

        # Only suggest rerouting if current lot is >90% full
        if current_occupancy_rate <= 0.9:
            return None

        competitors = self.find_competitors(features, all_spaces_data)

        # Find best alternative
        available_alternatives = [
            comp for comp in competitors 
            if comp['occupancy_rate'] < 0.7  # Less than 70% full
        ]

        if not available_alternatives:
            return None

        # Sort by distance (closest first)
        available_alternatives.sort(key=lambda x: x['distance'])

        best_alternative = available_alternatives[0]

        return {
            'recommended_space_id': best_alternative['space_id'],
            'distance': best_alternative['distance'],
            'occupancy_rate': best_alternative['occupancy_rate'],
            'price': best_alternative['price'],
            'reason': 'Current lot is over 90% full'
        }

class RealTimePricingEngine:
    """Main engine for real-time pricing simulation"""

    def __init__(self, data_loader: DataLoader):
        self.data_loader = data_loader
        self.models = {
            'linear': LinearPricingModel(),
            'demand_based': DemandBasedPricingModel(),
            'competitive': CompetitivePricingModel()
        }
        self.current_data = None
        self.results = []

    def process_real_time_data(self, delay_seconds: float = 1.0):
        """Simulate real-time data processing"""
        data = self.data_loader.processed_data

        if data is None:
            print("No data available for processing")
            return

        print("Starting real-time pricing simulation...")
        print(f"Processing {len(data)} records with {delay_seconds}s delay between updates")

        # Group by timestamp for real-time simulation
        grouped_data = data.groupby('timestamp' if 'timestamp' in data.columns else data.index)

        for timestamp, group in grouped_data:
            print(f"\nProcessing timestamp: {timestamp}")

            # Convert group to list of dictionaries for easier processing
            current_batch = group.to_dict('records')

            batch_results = []

            for record in current_batch:
                space_id = record['space_id']

                # Calculate prices using all models
                prices = {}
                recommendations = {}

                # Model 1: Linear
                prices['linear'] = self.models['linear'].calculate_price(record)
                self.models['linear'].update_price_history(space_id, prices['linear'], str(timestamp))

                # Model 2: Demand-based
                prices['demand_based'] = self.models['demand_based'].calculate_price(record)
                self.models['demand_based'].update_price_history(space_id, prices['demand_based'], str(timestamp))

                # Model 3: Competitive
                prices['competitive'] = self.models['competitive'].calculate_price(record, current_batch)
                self.models['competitive'].update_price_history(space_id, prices['competitive'], str(timestamp))

                # Check for rerouting recommendations
                rerouting = self.models['competitive'].suggest_rerouting(record, current_batch)

                result = {
                    'timestamp': timestamp,
                    'space_id': space_id,
                    'occupancy_rate': record['occupancy_rate'],
                    'queue_length': record['queue_length'],
                    'traffic_level': record.get('traffic_level', 0),
                    'is_special_day': record.get('is_special_day', False),
                    'vehicle_type': record.get('vehicle_type', 'car'),
                    'linear_price': prices['linear'],
                    'demand_based_price': prices['demand_based'],
                    'competitive_price': prices['competitive'],
                    'rerouting_suggestion': rerouting
                }

                batch_results.append(result)

            self.results.extend(batch_results)

            # Display current batch summary
            avg_prices = {
                'linear': np.mean([r['linear_price'] for r in batch_results]),
                'demand_based': np.mean([r['demand_based_price'] for r in batch_results]),
                'competitive': np.mean([r['competitive_price'] for r in batch_results])
            }

            print(f"Average prices - Linear: ${avg_prices['linear']:.2f}, "
                  f"Demand-based: ${avg_prices['demand_based']:.2f}, "
                  f"Competitive: ${avg_prices['competitive']:.2f}")

            # Simulate real-time delay
            if delay_seconds > 0:
                time.sleep(delay_seconds)

    def get_results_dataframe(self) -> pd.DataFrame:
        """Convert results to DataFrame"""
        return pd.DataFrame(self.results)

class PricingVisualizer:
    """Visualization component using Bokeh"""

    def __init__(self, results_df: pd.DataFrame):
        self.results_df = results_df

    def create_price_comparison_plot(self, space_ids: List[int] = None):
        """Create interactive price comparison plots"""
        if not BOKEH_AVAILABLE:
            print("Bokeh not available. Cannot create visualizations.")
            return None

        if space_ids is None:
            space_ids = sorted(self.results_df['space_id'].unique())[:4]  # Show first 4 spaces

        plots = []

        for space_id in space_ids:
            space_data = self.results_df[self.results_df['space_id'] == space_id].copy()
            space_data = space_data.sort_values('timestamp')

            # Create plot
            p = figure(
                title=f"Parking Space {space_id} - Dynamic Pricing",
                x_axis_label="Time",
                y_axis_label="Price ($)",
                width=800,
                height=300,
                x_axis_type='datetime'
            )

            # Convert timestamp to datetime if it's not already
            if not pd.api.types.is_datetime64_any_dtype(space_data['timestamp']):
                space_data['timestamp'] = pd.to_datetime(space_data['timestamp'])

            # Add lines for each model
            p.line(space_data['timestamp'], space_data['linear_price'], 
                   legend_label="Linear Model", line_color="blue", line_width=2)
            p.line(space_data['timestamp'], space_data['demand_based_price'], 
                   legend_label="Demand-Based Model", line_color="red", line_width=2)
            p.line(space_data['timestamp'], space_data['competitive_price'], 
                   legend_label="Competitive Model", line_color="green", line_width=2)

            # Add occupancy rate as secondary y-axis (scaled)
            p.line(space_data['timestamp'], space_data['occupancy_rate'] * 20, 
                   legend_label="Occupancy Rate (scaled)", line_color="orange", 
                   line_width=1, alpha=0.7)

            # Add hover tool
            hover = HoverTool(tooltips=[
                ("Time", "@x{%F %T}"),
                ("Linear Price", "$@y{0.00}"),
                ("Occupancy Rate", "@occupancy_rate{0.00}"),
                ("Queue Length", "@queue_length")
            ], formatters={'@x': 'datetime'})

            p.add_tools(hover)
            p.legend.location = "top_left"
            p.legend.click_policy = "hide"

            plots.append(p)

        # Create layout
        layout = column(*plots)

        # Save to HTML file
        output_file("parking_prices_visualization.html")
        save(layout)

        print("Visualization saved to 'parking_prices_visualization.html'")
        return layout

    def create_summary_statistics(self):
        """Create summary statistics"""
        print("\n" + "="*60)
        print("PRICING MODEL PERFORMANCE SUMMARY")
        print("="*60)

        # Overall statistics
        models = ['linear_price', 'demand_based_price', 'competitive_price']

        for model in models:
            model_name = model.replace('_price', '').replace('_', ' ').title()
            prices = self.results_df[model]

            print(f"\n{model_name} Model:")
            print(f"  Average Price: ${prices.mean():.2f}")
            print(f"  Price Range: ${prices.min():.2f} - ${prices.max():.2f}")
            print(f"  Standard Deviation: ${prices.std():.2f}")
            print(f"  Price Stability (CV): {prices.std()/prices.mean():.3f}")

        # Rerouting recommendations
        rerouting_count = self.results_df['rerouting_suggestion'].notna().sum()
        total_records = len(self.results_df)
        print(f"\nRerouting Recommendations: {rerouting_count}/{total_records} ({rerouting_count/total_records*100:.1f}%)")

        # Space-wise analysis
        print(f"\nSpace-wise Analysis:")
        space_stats = self.results_df.groupby('space_id').agg({
            'competitive_price': ['mean', 'std'],
            'occupancy_rate': 'mean',
            'queue_length': 'mean'
        }).round(2)

        print(space_stats.head(10))

def main():
    """Main execution function"""
    print(" DYNAMIC PRICING FOR URBAN PARKING LOTS")
    print("=" * 60)
    print("Multidisciplinary Expert Solution Framework")
    print("Real-time pricing engine with ML models built from scratch")
    print("=" * 60)

    # Initialize components
    data_loader = DataLoader()

    # Load data - will prompt user to select file
    data = data_loader.load_data()

    print(f" Data loaded successfully: {len(data)} records")
    print(f" Parking spaces: {data['space_id'].nunique()}")
    print(f" Date range: {data.get('timestamp', pd.Series(['N/A'])).min()} to {data.get('timestamp', pd.Series(['N/A'])).max()}")

    # Initialize pricing engine
    pricing_engine = RealTimePricingEngine(data_loader)

    # Process data with real-time simulation
    print("\n Starting real-time pricing simulation...")
    pricing_engine.process_real_time_data(delay_seconds=0.1)  # Fast simulation for demo

    # Get results
    results_df = pricing_engine.get_results_dataframe()

    if not results_df.empty:
        print(f" Processed {len(results_df)} pricing decisions")

        # Create visualizations
        visualizer = PricingVisualizer(results_df)
        visualizer.create_price_comparison_plot()
        visualizer.create_summary_statistics()

        # Save results
        results_df.to_csv("pricing_results.csv", index=False)
        print(f" Results saved to 'pricing_results.csv'")

        # Save model configurations
        model_config = {
            "base_price": 10.0,
            "linear_model_alpha": 0.5,
            "demand_model_params": {
                "alpha": 2.0, "beta": 1.5, "gamma": 0.5,
                "delta": 0.3, "epsilon": 0.2, "lambda": 0.8
            },
            "competitive_radius": 0.01,
            "timestamp": datetime.now().isoformat()
        }

        with open("model_configuration.json", "w") as f:
            json.dump(model_config, f, indent=2)

        print(" Model configuration saved to 'model_configuration.json'")
    else:
        print(" No results generated")

    print("\n" + "="*60)
    print("SOLUTION DELIVERY COMPLETE")
    print("="*60)
    print("Three pricing models implemented (Linear, Demand-based, Competitive)")
    print("Real-time simulation with streaming data processing")
    print("Interactive visualizations with Bokeh")
    print("Rerouting recommendations for overburdened lots")
    print("Comprehensive performance analysis")
    print("Academic-grade documentation and reproducibility")
    print("\n Files generated:")
    print("  - parking_prices_visualization.html (Interactive plots)")
    print("  - pricing_results.csv (All pricing decisions)")
    print("  - model_configuration.json (Model parameters)")

if __name__ == "__main__":
    main()


SyntaxError: expected 'except' or 'finally' block (4076844263.py, line 20)